In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as ww
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time
import re

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.donga.com/news/List?p={}1&prod=news&ymd=&m="
article_base = "https://www.donga.com/"

dict = defaultdict(list)
for page in range(0, 2, 2):
    url = base_url.format(page)
    driver.get(url)
    time.sleep(2)

    # 기사 제목과 링크 추출
    
    li_list = driver.find_elements(By.CSS_SELECTOR, 'ul.row_list > li')
    for li in li_list:
        try:
            a_tag = li.find_element(By.CSS_SELECTOR, 'h4.tit > a')
            href = a_tag.get_attribute('href')
            print(href)
        except:
            continue  # 해당 li는 건너뜀



        # open new window with article page
        driver.execute_script('window.open(arguments[0]);', href)
        driver.switch_to.window(driver.window_handles[1])

        article_soup = bs(driver.page_source, 'html.parser')

        element = article_soup.select_one('#contents > header > div > section > nav > ol > li > a')
        if element:
            dict['Category'].append(element.get_text(strip=True))
        else:
            dict['Category'].append('')

        dict['Title'].append(article_soup.select_one('#contents > header > div > section > h1').get_text(strip=True))
        

        date_element = article_soup.select_one('#contents > header > div > section > ul > li:nth-child(2) > button > span:nth-child(1)')
        if date_element:
            date_text = date_element.get_text(strip=True).split()[0]
            dict['Date'].append(date_text)
        else:
            dict['Date'].append('날짜 없음')


        content = article_soup.select_one('#contents > div.view_body > div > div.main_view > section.news_view')
        if content:
            text = content.get_text(separator=' ', strip=True)
            dict['Contents'].append(text)
    


        driver.close()
        driver.switch_to.window(driver.window_handles[0])
driver.quit()

print(dict)


https://www.donga.com/news/Society/article/all/20251127/132855569/1
https://www.donga.com/news/Culture/article/all/20251127/132856348/2
https://www.donga.com/news/Society/article/all/20251127/132855847/1
https://www.donga.com/news/Politics/article/all/20251127/132856796/2
https://www.donga.com/news/It/article/all/20251127/132855679/1
https://www.donga.com/news/Society/article/all/20251127/132856783/1
https://www.donga.com/news/Society/article/all/20251127/132855860/2
https://www.donga.com/news/Society/article/all/20251127/132856741/2
https://www.donga.com/news/Inter/article/all/20251127/132855203/1
https://www.donga.com/news/Politics/article/all/20251127/132856640/1
https://www.donga.com/news/Society/article/all/20251127/132856597/2
https://www.donga.com/news/Society/article/all/20251127/132856607/1
https://www.donga.com/news/Inter/article/all/20251127/132856581/1
https://www.donga.com/news/Society/article/all/20251127/132856569/1
https://www.donga.com/news/Entertainment/article/all/20

In [32]:
import json

articles = []
for i in range(len(dict['Category'])):
    article = {
        "num": i+1,
        "Category": dict['Category'][i],
        "Date": dict['Date'][i],
        "Title": dict['Title'][i],
        "Contents": dict['Contents'][i]
    }
    articles.append(article)

# JSON으로 저장
with open('articles_donga.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)